<a href="https://colab.research.google.com/github/HaoyuHunterFang/schulich_data_science/blob/main/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'stanford_dogs',
    split = ['train','test'],
    with_info = True,
    as_supervised = True
)

ds_train = ds_train.take(int(0.8*len(ds_train)))
ds_validation = ds_train.skip(int(0.8*len(ds_train)))

In [3]:
#Checking the size of train and test datasets
print(ds_info.splits)

{'train': <SplitInfo num_examples=12000, num_shards=4>, 'test': <SplitInfo num_examples=8580, num_shards=4>}


In [4]:
def preprocess(image, label):
  image = tf.image.resize(image, (224,224))
  image = tf.cast(image, tf.float32) / 255.0
  label = tf.one_hot(label, depth = ds_info.features['label'].num_classes)
  return image, label

ds_train = ds_train.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
ds_validation = ds_validation.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [5]:
# custom model

custom_model = tf.keras.Sequential([
    #32 is Random no
    tf.keras.layers.Conv2D(32,(3,3), activation='relu', input_shape = (224,224,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3),activation = 'relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(ds_info.features['label'].num_classes, activation = 'softmax')
])

custom_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

# New Section

In [6]:
custom_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 flatten (Flatten)           (None, 346112)            0

In [ ]:
custom_model_history = custom_model.fit(
    ds_train,
    epochs = 10,
    validation_data = ds_validation,
    batch_size = 64
)

Epoch 1/10
300/300 [==============================] - 53s 125ms/step - loss: 4.8686 - accuracy: 0.0083 - val_loss: 4.7564 - val_accuracy: 0.0833
Epoch 2/10
300/300 [==============================] - 39s 129ms/step - loss: 4.7080 - accuracy: 0.0214 - val_loss: 4.2782 - val_accuracy: 0.2073
Epoch 3/10
300/300 [==============================] - 38s 126ms/step - loss: 3.7535 - accuracy: 0.2099 - val_loss: 2.1529 - val_accuracy: 0.6724
Epoch 4/10
300/300 [==============================] - 36s 119ms/step - loss: 2.3039 - accuracy: 0.5215 - val_loss: 0.9376 - val_accuracy: 0.7974
Epoch 5/10
 58/300 [====>.........................] - ETA: 21s - loss: 1.8126 - accuracy: 0.6374

In [ ]:
test_loss, test_accuracy = custom_model.evaluate(ds_test)
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
def augment(image1, label1):
    image1 = tf.image.random_flip_left_right(image1)
    image1 = tf.image.random_brightness(image1, max_delta = 0.1)
    image1 = tf.image.random_contrast(image1,lower=0.8,upper=1.2)
    image1 = tf.image.random_saturation(image1,lower = 0.5, upper = 1.5)
    image1 = tf.image.random_hue(image1,max_delta=0.2)
    image1 = tf.image.random_flip_up_down(image1)
    label1 = tf.one_hot(label1, depth = ds_info.features['label'].num_classes)
    return image1, label1

In [ ]:
(ds_train_1, ds_test_1), ds_info_1 = tfds.load(
    'stanford_dogs',
    split = ['train','test'],
    with_info = True,
    as_supervised = True
)

ds_train_1 = ds_train_1.take(int(0.8*len(ds_train_1)))
ds_validation_1 = ds_train_1.skip(int(0.8*len(ds_train_1)))

In [ ]:
ds_train_1 = ds_train_1.map(augment)

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train_1 = ds_train.cache().shuffle(1000).batch(64).prefetch(buffer_size=AUTOTUNE)
ds_validation_1 = ds_validation.batch(64).prefetch(buffer_size=AUTOTUNE)

In [ ]:
custom_model.fit(
    ds_train_1,
    epochs=20,
    validation_data=ds_validation_1,
    callbacks = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True)
)